In [1]:
import numpy as np
# Nu trebuie tf pentru citirea datelor, dar trebuie pentru tot restul
import torch 
def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784])
    mnist_train_labels = np.zeros(60000)
    
    f = open('train-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('train-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,784)
    mnist_train_data = mnist_train_data.reshape(60000,1,28,28)
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
    
    return mnist_train_data, mnist_train_labels

def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open('t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784)
    mnist_test_data = mnist_test_data.reshape(10000,1,28,28)
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)
    

In [6]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class Retea_MLP(nn.Module):
    
    def __init__(self, nr_neuroni_input, nr_neuroni_hidden, nr_clase):
        
        # Pentru a putea folosi mai departe reteaua, este recomandata mostenirea
        # clasei de baza nn.Module
        super(Retea_MLP,self).__init__()
        
        # Definirea ponderilor si a deplasamentelor din stratul ascuns
        self.w_h = torch.randn(nr_neuroni_input, nr_neuroni_hidden, dtype = torch.float, requires_grad=True)
        self.b_h = torch.randn(nr_neuroni_hidden, dtype = torch.float, requires_grad=True)
        
        # Definirea ponderilor si a deplasamentelor din stratul de iesire
        self.w_o = torch.randn(nr_neuroni_hidden, nr_clase, dtype = torch.float, requires_grad=True)
        self.b_o = torch.randn(nr_clase, dtype = torch.float, requires_grad=True)

        self.relu = nn.ReLU()
        
    
    # Se aduna toate variabilele antrenabile intr-o lista, pentru a putea face referire rapida la ele
    def parameters(self):
        return [self.w_h, self.b_h, self.w_o, self.b_o]
    
    def forward(self,input_batch):
        # Intr-un MLP, intrarea este sub forma unui vector, deci un batch
        # este o matrice de dimensiunea nr_esantioane_batch x dimensiune esantion
        input_batch = torch.from_numpy(input_batch)
        self.hidden = torch.mm(input_batch, self.w_h) + self.b_h

        self.hidden = self.relu(self.hidden)
        
        out = torch.mm(self.hidden, self.w_o) + self.b_o

        # out = softmax(out) - se poate si fara (softmax este inclus in CrossEntropyLoss)
        
        return out

In [26]:
class Retea_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definirea straturilor CNN
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        
        self.relu = nn.ReLU()

        # Definirea stratului fully connected
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128,10)
        
    def forward(self, x):

        # Definirea propagarii forward
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))

        x = x.view(x.size(0), -1)  # Flattening

        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
from matplotlib import pyplot as plt


def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784])
    mnist_train_labels = np.zeros(60000)
    
    f = open('train-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('train-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,784)
    mnist_train_data = mnist_train_data.reshape(60000,1,28,28)
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
    
    return mnist_train_data, mnist_train_labels

def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open('t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784)
    mnist_test_data = mnist_test_data.reshape(10000,1,28,28)
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)
    
    return mnist_test_data, mnist_test_labels



class DatasetMNIST(Dataset):
    def __init__(self):

        mnist_train_data,mnist_train_labels = get_MNIST_train()

        self.data = mnist_train_data
        self.labels = mnist_train_labels

    def __getitem__(self, index):
        data_point = self.data[index, ...]
        label_point = self.labels[index, ...]

        return {'data': data_point, 'label': label_point}
    
    def __len__(self):
        return self.data.shape[0]

mnistdataset = DatasetMNIST()

mnistdataloader = DataLoader(mnistdataset, batch_size=128, shuffle=True, num_workers=0)

loss_function = nn.CrossEntropyLoss(reduction='sum')

simple_CNN = Retea_CNN().to("cuda")

optim = torch.optim.SGD(simple_CNN.parameters(), lr=1e-5)

for ep in range(15):

    predictii = []
    etichete = []

    for batch in mnistdataloader:
        data = batch['data'].to("cuda")
        label = batch['label'].to("cuda")

        # print(data.shape)
        # print(label.shape)

        # data_as_image = data[0, ...].cpu().detach().numpy().squeeze()
        # print(data_as_image.shape)

        # plt.imshow(data_as_image, cmap='gray')
        # continue

        current_predict = simple_CNN.forward(data)

        # Se calculeaza valoarea momentana a functiei loss
        loss = loss_function(current_predict, label) 
        
        # Se memoreaza predictiile si etichetele aferente batch-ului actual (pentru calculul acuratetii)
        current_predict = np.argmax(current_predict.cpu().detach().numpy(), axis=1)
        label = label.cpu().detach().numpy()

        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,label))

        optim.zero_grad()
            # 2. Calculul tuturor gradientilor. Backpropagation
        loss.backward()
            # 3. Actualizarea tuturor ponderilor, pe baza gradientilor.
        optim.step()
        
        

    # Calculam acuratetea
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )





Acuratetea la epoca 1 este 92.04833333333333%
Acuratetea la epoca 2 este 97.07666666666667%
Acuratetea la epoca 3 este 97.82166666666666%
Acuratetea la epoca 4 este 98.22166666666666%
Acuratetea la epoca 5 este 98.50666666666666%
Acuratetea la epoca 6 este 98.68333333333334%
Acuratetea la epoca 7 este 98.80333333333333%
Acuratetea la epoca 8 este 99.00666666666666%
Acuratetea la epoca 9 este 99.12166666666667%
Acuratetea la epoca 10 este 99.16%
Acuratetea la epoca 11 este 99.26166666666667%
Acuratetea la epoca 12 este 99.31333333333333%
Acuratetea la epoca 13 este 99.385%
Acuratetea la epoca 14 este 99.47333333333333%
Acuratetea la epoca 15 este 99.49166666666667%
